In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 54kB/s 
     |████████████████████████████████| 204kB 41.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=6cc110f4caf2fdf1f732aa67eaa8705cc0a314283b5a5b6fbecc3fd8dde19153
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 43 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

## Segundo Cuatrimestre de 2018.Examen parcial, tercera oportunidad

### 1)La empresa Instagram almacena informacion de usuarios en un RDD de la forma (id_usuario, nickname, pais ,fecha_alta) y otro RDD almacena informacion de las publicaciones como (id_publicacion, id_usuario, id_foto, comentario, fecha). En Instagram, un hastag es una palabra que comienza con 
### Se pide realizar lo siguiente utilizando el api de RDD en pyspark:

### a) Encontrar el top 10 de hastags que mas aparecen y que fueron publicados unicamente por ususarios del pais Argentina.(Por ejemplo: si un hashtag fue publicado por alguien que no es de Argentina entonces no hay que consisderar dicho hashtag para el top 10, aunque sea el que mayor cantidad de ocurrencias haya acumulado).

### b) Construir y cachear un RDD con los pares de usuarios (id_usuario1, id_usuario2) que tienen hashtags publicados en comun

In [17]:
publicaciones_doc = [(16747,9838,116,'Every man #chabon has his #spiderman price','2019/03/11'),
(36922,1438,126,'Youve got to #locura ask yourself one question Do I feel lucky? Well do ya punk','2019/03/29'),
(11386,1592,108,'Like two peas #locura #chabon in a pod','2019/01/28'),
(39223,1082,137,'Dont put the cart before the horse','2019/03/02'),
(8296,2639,106,'Nostalgia isnt what it used to be','2019/03/02'),
(16116,3829,137,'Shes drop dead #chabon gorgeous','2019/01/11'),
(20129,7677,146,'Too many #dromedario cooks spoil the broth','2019/01/04'),
(44960,8276,140,'If you cant beat them, join them','2019/01/28'),
(6832,3536,148,'I like turtles','2019/01/11'),
(44692,6709,150,'Ask no questions and hear no lies','2019/02/27'),
(12374,1328,122,'A bird in the #locura hand is #chabon worth two in the bush','2019/03/01'),
(50730,3964,148,'Youll #dromedario poke your eye out','2019/01/09'),
(47049,1880,122,'You can #bobsponge lead a horse #chabon to water but you cant make him drink','2019/02/16'),
(6320,5624,127,'Two heads are better than one','2019/01/23'),
(37911,1444,132,'Im going to make #chabon him an offer he cant refuse','2019/02/17'),
(44080,4014,113,'A sight for sore eyes','2019/02/12'),
(57898,5635,118,'Theres #bobsponge no place #dromedario like home','2019/01/01'),
(10721,5123,142,'Absence #dromedario makes the heart #locura grow fonder','2019/01/24'),
(57053,6084,116,'Youre barking up the wrong tree','2019/02/01'),
(11695,2885,141,'Old habits die hard','2019/02/07'),
(22564,6943,145,'No pain no gain','2019/03/09'),
(21307,1813,136,'Variety is the spice of #chabon #locura life','2019/03/10'),
(49413,1554,103,'Would you jump off #bobsponge a bridge if all your friends were doing it','2019/03/13'),
(55044,3591,129,'A friend in need is a friend indeed','2019/04/01'),
(38938,8740,137,'Dont cry over spilled milk','2019/01/01'),
(31140,9651,106,'The needs of the #bobsponge many #dromedario outweigh the needs #locura of the few','2019/02/05'),
(44856,6027,136,'Because I said so, thats #dromedario why','2019/03/13'),
(4517,9673,103,'The Tribe has spoken','2019/02/15'),
(28579,4650,150,'The pen is mightier #bobsponge than the sword','2019/01/18'),
(52683,6501,119,'Leave well enough #dromedario alone','2019/01/20')]

usuarios_doc = [(9838,'Raindrop','argentina','5/14/2018'),
(1438,'Admiral','uruguay','9/24/2018'),
(1592,'Mustache','uruguay','7/7/2018'),
(1082,'Bean','argentina','2/9/2018'),
(2639,'Fatty','uruguay','4/18/2018'),
(3829,'Queenie','uruguay','1/23/2018'),
(7677,'Sugar','argentina','8/7/2018'),
(8276,'Righty','argentina','10/10/2018'),
(3536,'Fellow','argentina','9/3/2018'),
(6709,'Dearest','argentina','8/6/2018'),
(1328,'Salt','uruguay','5/11/2018'),
(3964,'Belch','argentina','8/28/2018'),
(1880,'Boo Bear','uruguay','5/23/2018'),
(5624,'Bello','argentina','2/6/2018'),
(1444,'Chain','argentina','11/1/2018'),
(4014,'Doc','argentina','7/4/2018'),   
(5635,'Dirty Harry','argentina','4/5/2018'),
(5123,'Jackrabbit','argentina','5/31/2018'),
(6084,'Foxy','uruguay','11/4/2018'),
(2885,'Cruella','argentina','11/25/2018'),
(6943,'Guy','uruguay','10/1/2018'),
(1813,'Buck','chile','8/4/2018'),
(1554,'Peppermint','chile','2/15/2018'),
(3591,'Headlights','chile','1/6/2018'),
(8740,'Slim','chile','10/22/2018'),
(9651,'Skunk','chile','8/18/2018'),
(6027,'Blondie','chile','9/16/2018'),
(9673,'Bandit','argentina','4/11/2018'),
(4650,'Sport','argentina','11/7/2018'),
(6501,'Pookie','argentina','11/7/2018')]


In [18]:
publicacionesRDD = sc.parallelize(publicaciones_doc)
usuariosRDD = sc.parallelize(usuarios_doc)

In [19]:
publicacionesRDD.collect()

[(16747,
  9838,
  116,
  'Every man #chabon has his #spiderman price',
  '2019/03/11'),
 (36922,
  1438,
  126,
  'Youve got to #locura ask yourself one question Do I feel lucky? Well do ya punk',
  '2019/03/29'),
 (11386, 1592, 108, 'Like two peas #locura #chabon in a pod', '2019/01/28'),
 (39223, 1082, 137, 'Dont put the cart before the horse', '2019/03/02'),
 (8296, 2639, 106, 'Nostalgia isnt what it used to be', '2019/03/02'),
 (16116, 3829, 137, 'Shes drop dead #chabon gorgeous', '2019/01/11'),
 (20129,
  7677,
  146,
  'Too many #dromedario cooks spoil the broth',
  '2019/01/04'),
 (44960, 8276, 140, 'If you cant beat them, join them', '2019/01/28'),
 (6832, 3536, 148, 'I like turtles', '2019/01/11'),
 (44692, 6709, 150, 'Ask no questions and hear no lies', '2019/02/27'),
 (12374,
  1328,
  122,
  'A bird in the #locura hand is #chabon worth two in the bush',
  '2019/03/01'),
 (50730, 3964, 148, 'Youll #dromedario poke your eye out', '2019/01/09'),
 (47049,
  1880,
  122,
  'You

In [20]:
usuariosRDD.collect()

[(9838, 'Raindrop', 'argentina', '5/14/2018'),
 (1438, 'Admiral', 'uruguay', '9/24/2018'),
 (1592, 'Mustache', 'uruguay', '7/7/2018'),
 (1082, 'Bean', 'argentina', '2/9/2018'),
 (2639, 'Fatty', 'uruguay', '4/18/2018'),
 (3829, 'Queenie', 'uruguay', '1/23/2018'),
 (7677, 'Sugar', 'argentina', '8/7/2018'),
 (8276, 'Righty', 'argentina', '10/10/2018'),
 (3536, 'Fellow', 'argentina', '9/3/2018'),
 (6709, 'Dearest', 'argentina', '8/6/2018'),
 (1328, 'Salt', 'uruguay', '5/11/2018'),
 (3964, 'Belch', 'argentina', '8/28/2018'),
 (1880, 'Boo Bear', 'uruguay', '5/23/2018'),
 (5624, 'Bello', 'argentina', '2/6/2018'),
 (1444, 'Chain', 'argentina', '11/1/2018'),
 (4014, 'Doc', 'argentina', '7/4/2018'),
 (5635, 'Dirty Harry', 'argentina', '4/5/2018'),
 (5123, 'Jackrabbit', 'argentina', '5/31/2018'),
 (6084, 'Foxy', 'uruguay', '11/4/2018'),
 (2885, 'Cruella', 'argentina', '11/25/2018'),
 (6943, 'Guy', 'uruguay', '10/1/2018'),
 (1813, 'Buck', 'chile', '8/4/2018'),
 (1554, 'Peppermint', 'chile', '2/15/

In [21]:
#Nos quedamos unicamente con los campos del RDD necesarios
publicaciones = publicacionesRDD.map(lambda x: (x[1],x[3]))
publicaciones.collect()

[(9838, 'Every man #chabon has his #spiderman price'),
 (1438,
  'Youve got to #locura ask yourself one question Do I feel lucky? Well do ya punk'),
 (1592, 'Like two peas #locura #chabon in a pod'),
 (1082, 'Dont put the cart before the horse'),
 (2639, 'Nostalgia isnt what it used to be'),
 (3829, 'Shes drop dead #chabon gorgeous'),
 (7677, 'Too many #dromedario cooks spoil the broth'),
 (8276, 'If you cant beat them, join them'),
 (3536, 'I like turtles'),
 (6709, 'Ask no questions and hear no lies'),
 (1328, 'A bird in the #locura hand is #chabon worth two in the bush'),
 (3964, 'Youll #dromedario poke your eye out'),
 (1880,
  'You can #bobsponge lead a horse #chabon to water but you cant make him drink'),
 (5624, 'Two heads are better than one'),
 (1444, 'Im going to make #chabon him an offer he cant refuse'),
 (4014, 'A sight for sore eyes'),
 (5635, 'Theres #bobsponge no place #dromedario like home'),
 (5123, 'Absence #dromedario makes the heart #locura grow fonder'),
 (6084, '

In [22]:
#Obtenemos los usuarios no argentinos
usuarios_noArgentinos = (usuariosRDD.map(lambda x: (x[0],x[2]))).filter(lambda x: x[1]!='argentina')
usuarios_noArgentinos.collect()

[(1438, 'uruguay'),
 (1592, 'uruguay'),
 (2639, 'uruguay'),
 (3829, 'uruguay'),
 (1328, 'uruguay'),
 (1880, 'uruguay'),
 (6084, 'uruguay'),
 (6943, 'uruguay'),
 (1813, 'chile'),
 (1554, 'chile'),
 (3591, 'chile'),
 (8740, 'chile'),
 (9651, 'chile'),
 (6027, 'chile')]

In [23]:
# Unificamos las publicaciones con los usuarios no argentinos
hashtags_noArgentinos = (usuarios_noArgentinos.join(publicaciones)).map(lambda x: (x[0],x[1][1]))
hashtags_noArgentinos.collect()

[(1592, 'Like two peas #locura #chabon in a pod'),
 (1328, 'A bird in the #locura hand is #chabon worth two in the bush'),
 (1880,
  'You can #bobsponge lead a horse #chabon to water but you cant make him drink'),
 (6084, 'Youre barking up the wrong tree'),
 (8740, 'Dont cry over spilled milk'),
 (3829, 'Shes drop dead #chabon gorgeous'),
 (1813, 'Variety is the spice of #chabon #locura life'),
 (1438,
  'Youve got to #locura ask yourself one question Do I feel lucky? Well do ya punk'),
 (1554,
  'Would you jump off #bobsponge a bridge if all your friends were doing it'),
 (2639, 'Nostalgia isnt what it used to be'),
 (6943, 'No pain no gain'),
 (3591, 'A friend in need is a friend indeed'),
 (9651,
  'The needs of the #bobsponge many #dromedario outweigh the needs #locura of the few'),
 (6027, 'Because I said so, thats #dromedario why')]

In [24]:
#Nos quedamos unicamente con las publicaciones que tienen hashtags
tweet_con_hasgtags_noArgentinos = hashtags_noArgentinos.filter(lambda x: '#' in x[1] )
tweet_con_hasgtags_noArgentinos.collect()

[(1592, 'Like two peas #locura #chabon in a pod'),
 (1328, 'A bird in the #locura hand is #chabon worth two in the bush'),
 (1880,
  'You can #bobsponge lead a horse #chabon to water but you cant make him drink'),
 (3829, 'Shes drop dead #chabon gorgeous'),
 (1813, 'Variety is the spice of #chabon #locura life'),
 (1438,
  'Youve got to #locura ask yourself one question Do I feel lucky? Well do ya punk'),
 (1554,
  'Would you jump off #bobsponge a bridge if all your friends were doing it'),
 (9651,
  'The needs of the #bobsponge many #dromedario outweigh the needs #locura of the few'),
 (6027, 'Because I said so, thats #dromedario why')]

In [25]:
# Definimos una funcion que atrapa los hashtags en cada publicacion
def hashtagFetcher (x):
  listaHashtags = []
  for palabra in (x[1].split()):
    if ('#' in palabra):
          listaHashtags.append(palabra)
  return listaHashtags

In [26]:
# Obtenemos la lista de hashtags que ocurren en las publicaciones no argentinas
hashtags_por_tweet = tweet_con_hasgtags_noArgentinos.map(lambda x: (x[0], hashtagFetcher(x)))
hashtags_lista = (hashtags_por_tweet.flatMap(lambda x: (x[1]))).collect()
hashtags_lista
hashtags_lista_sin_repetir = set(list(hashtags_lista))
hashtags_lista_sin_repetir

{'#bobsponge', '#chabon', '#dromedario', '#locura'}

In [27]:
# Generamos un RDD con los tweets con hashtags y nos quedamos solo con los hashtags
publicaciones_hashtags = (publicacionesRDD.flatMap(lambda x: [(word,x[1]) for word in (x[3].split())])).filter(lambda x: '#' in x[0]).map(lambda x: (x[1], (x[0],1)))
publicaciones_hashtags.collect()

[(9838, ('#chabon', 1)),
 (9838, ('#spiderman', 1)),
 (1438, ('#locura', 1)),
 (1592, ('#locura', 1)),
 (1592, ('#chabon', 1)),
 (3829, ('#chabon', 1)),
 (7677, ('#dromedario', 1)),
 (1328, ('#locura', 1)),
 (1328, ('#chabon', 1)),
 (3964, ('#dromedario', 1)),
 (1880, ('#bobsponge', 1)),
 (1880, ('#chabon', 1)),
 (1444, ('#chabon', 1)),
 (5635, ('#bobsponge', 1)),
 (5635, ('#dromedario', 1)),
 (5123, ('#dromedario', 1)),
 (5123, ('#locura', 1)),
 (1813, ('#chabon', 1)),
 (1813, ('#locura', 1)),
 (1554, ('#bobsponge', 1)),
 (9651, ('#bobsponge', 1)),
 (9651, ('#dromedario', 1)),
 (9651, ('#locura', 1)),
 (6027, ('#dromedario', 1)),
 (4650, ('#bobsponge', 1)),
 (6501, ('#dromedario', 1))]

In [29]:
final_final = (publicaciones_hashtags.filter(lambda x: x[1][0]  not in hashtags_lista)).map(lambda x: (x[1][0],x[0]))
final_final.collect()

[('#spiderman', 9838)]

b) Construir y cachear un RDD con los pares de usuarios (id_usuario1, id_usuario2) que tienen hashtags publicados en comun 

In [ ]:
) 

In [34]:
pares_aux = (publicacionesRDD.flatMap(lambda x: [(word,x[1]) for word in (x[3].split())])).filter(lambda x: '#' in x[0])
pares_usuarios_hashtags = ((pares_aux.join(pares_aux)).filter(lambda x: x[1][0]!=x[1][1])).map(lambda x: (x[1][0],x[1][1]))
pares_usuarios_hashtags.collect()

[(1438, 1592),
 (1438, 1328),
 (1438, 5123),
 (1438, 1813),
 (1438, 9651),
 (1592, 1438),
 (1592, 1328),
 (1592, 5123),
 (1592, 1813),
 (1592, 9651),
 (1328, 1438),
 (1328, 1592),
 (1328, 5123),
 (1328, 1813),
 (1328, 9651),
 (5123, 1438),
 (5123, 1592),
 (5123, 1328),
 (5123, 1813),
 (5123, 9651),
 (1813, 1438),
 (1813, 1592),
 (1813, 1328),
 (1813, 5123),
 (1813, 9651),
 (9651, 1438),
 (9651, 1592),
 (9651, 1328),
 (9651, 5123),
 (9651, 1813),
 (1880, 5635),
 (1880, 1554),
 (1880, 9651),
 (1880, 4650),
 (5635, 1880),
 (5635, 1554),
 (5635, 9651),
 (5635, 4650),
 (1554, 1880),
 (1554, 5635),
 (1554, 9651),
 (1554, 4650),
 (9651, 1880),
 (9651, 5635),
 (9651, 1554),
 (9651, 4650),
 (4650, 1880),
 (4650, 5635),
 (4650, 1554),
 (4650, 9651),
 (9838, 1592),
 (9838, 3829),
 (9838, 1328),
 (9838, 1880),
 (9838, 1444),
 (9838, 1813),
 (1592, 9838),
 (1592, 3829),
 (1592, 1328),
 (1592, 1880),
 (1592, 1444),
 (1592, 1813),
 (3829, 9838),
 (3829, 1592),
 (3829, 1328),
 (3829, 1880),
 (3829, 14

# FIN DEL EJERCICIO